In [1]:
import pandas as pd
import numpy as np
import pyreadr

from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters
from tsfresh.utilities.dataframe_functions import roll_time_series, impute

/Users/aphil/opt/anaconda3/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
result = pyreadr.read_r('Data/M5dataset.Rdata')

In [3]:
result.keys()

odict_keys(['calendar', 'trainset', 'testset', '.Random.seed', 'prices', 'info'])

In [4]:
calendar = result['calendar']
trainset = result['trainset']
testset = result['testset']
random_seed = result['.Random.seed']
prices = result['prices']
info = result['info']

In [5]:
data = pd.concat([trainset, testset.iloc[:,5:]], axis=1)

## Select only top 10 foods items

In [6]:
data["id"] = data.index

In [7]:
data =pd.wide_to_long(data, stubnames='d_', i= ['id'], j='day')

In [8]:
data = data.reset_index()

In [9]:
# Select only foods
data = data[data["cat_id"]=="FOODS"]

In [10]:
data = data.drop(["dept_id", "id", "cat_id"], axis=1)

In [11]:
data

,day,state_id,store_id,item_id,d_
1612,1,CA,CA_1,FOODS_1_001,3
1613,1,CA,CA_1,FOODS_1_002,0
1614,1,CA,CA_1,FOODS_1_003,0
1615,1,CA,CA_1,FOODS_1_004,0
1616,1,CA,CA_1,FOODS_1_005,3
...,...,...,...,...,...
60034805,1969,WI,WI_3,FOODS_3_823,0
60034806,1969,WI,WI_3,FOODS_3_824,0
60034807,1969,WI,WI_3,FOODS_3_825,0
60034808,1969,WI,WI_3,FOODS_3_826,5


In [12]:
data.rename(columns={'d_':'demand'}, inplace=True)

In [13]:
non_zero = data[data["demand"]!=0]
non_zero_agg = non_zero.groupby(['item_id'])
size = non_zero_agg.size()
size = size.sort_values(ascending=False)
size = pd.DataFrame(size).reset_index()

In [14]:
size

,item_id,0
0,FOODS_3_586,19602
1,FOODS_3_694,19561
2,FOODS_3_080,19548
3,FOODS_3_555,19530
4,FOODS_3_714,19522
...,...,...
1432,FOODS_3_296,966
1433,FOODS_1_079,946
1434,FOODS_2_073,930
1435,FOODS_3_472,893


In [15]:
top_products = size.head(10)["item_id"]

In [16]:
data = data[data["item_id"].isin(top_products)]

In [17]:
data

,day,state_id,store_id,item_id,demand
2304,1,CA,CA_1,FOODS_3_080,33
2450,1,CA,CA_1,FOODS_3_226,13
2452,1,CA,CA_1,FOODS_3_228,14
2476,1,CA,CA_1,FOODS_3_252,19
2601,1,CA,CA_1,FOODS_3_377,20
...,...,...,...,...,...
60034540,1969,WI,WI_3,FOODS_3_555,39
60034571,1969,WI,WI_3,FOODS_3_586,38
60034653,1969,WI,WI_3,FOODS_3_668,12
60034679,1969,WI,WI_3,FOODS_3_694,49


## Add calendar features

In [18]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,NaN,NaN,NaN,NaN,0,0,0


In [19]:
calendar.reset_index(inplace=True)

In [20]:
calendar['index'] = calendar['index']+1

### Preprocess events into OHE

In [21]:
# add indicator if there is an event on that day
calendar['is_event'] = 1-(calendar.event_name_1.isnull())

In [22]:
#add indicator of the type of event
calendar['is_sporting_event'] = ((calendar.event_type_1=='Sporting')|(calendar.event_type_2=='Sporting')).astype(int)
calendar['is_cultural_event'] = ((calendar.event_type_1=='Cultural')|(calendar.event_type_2=='Cultural')).astype(int)
calendar['is_national_event'] = ((calendar.event_type_1=='National')|(calendar.event_type_2=='National')).astype(int)
calendar['is_religious_event'] = ((calendar.event_type_1=='Religious')|(calendar.event_type_2=='Religious')).astype(int)

In [23]:
calendar

,index,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_event,is_sporting_event,is_cultural_event,is_national_event,is_religious_event
0,1,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
1,2,2011-01-30,11101,Sunday,2,1,2011,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
2,3,2011-01-31,11101,Monday,3,1,2011,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
3,4,2011-02-01,11101,Tuesday,4,2,2011,NaN,NaN,NaN,NaN,1,1,0,0,0,0,0,0
4,5,2011-02-02,11101,Wednesday,5,2,2011,NaN,NaN,NaN,NaN,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,1965,2016-06-15,11620,Wednesday,5,6,2016,NaN,NaN,NaN,NaN,0,1,1,0,0,0,0,0
1965,1966,2016-06-16,11620,Thursday,6,6,2016,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
1966,1967,2016-06-17,11620,Friday,7,6,2016,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
1967,1968,2016-06-18,11621,Saturday,1,6,2016,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0


### Merge 

In [24]:
data = data.merge(calendar.loc[:,['index', 'date', 'weekday', 'wday', 'month', 'year', 'is_sporting_event',
       'is_cultural_event', 'is_national_event', 'is_religious_event']], left_on='day', right_on='index')

In [25]:
data

,day,state_id,store_id,item_id,demand,index,date,weekday,wday,month,year,is_sporting_event,is_cultural_event,is_national_event,is_religious_event
0,1,CA,CA_1,FOODS_3_080,33,1,2011-01-29,Saturday,1,1,2011,0,0,0,0
1,1,CA,CA_1,FOODS_3_226,13,1,2011-01-29,Saturday,1,1,2011,0,0,0,0
2,1,CA,CA_1,FOODS_3_228,14,1,2011-01-29,Saturday,1,1,2011,0,0,0,0
3,1,CA,CA_1,FOODS_3_252,19,1,2011-01-29,Saturday,1,1,2011,0,0,0,0
4,1,CA,CA_1,FOODS_3_377,20,1,2011-01-29,Saturday,1,1,2011,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196895,1969,WI,WI_3,FOODS_3_555,39,1969,2016-06-19,Sunday,2,6,2016,1,1,0,0
196896,1969,WI,WI_3,FOODS_3_586,38,1969,2016-06-19,Sunday,2,6,2016,1,1,0,0
196897,1969,WI,WI_3,FOODS_3_668,12,1969,2016-06-19,Sunday,2,6,2016,1,1,0,0
196898,1969,WI,WI_3,FOODS_3_694,49,1969,2016-06-19,Sunday,2,6,2016,1,1,0,0


### Pre-processing

In [26]:
# select columns
data = data[["index", "wday", "month", "year", "item_id", "store_id", "state_id", "is_sporting_event", "is_cultural_event", "is_national_event", "is_religious_event", "demand"]]

In [27]:
data

,index,wday,month,year,item_id,store_id,state_id,is_sporting_event,is_cultural_event,is_national_event,is_religious_event,demand
0,1,1,1,2011,FOODS_3_080,CA_1,CA,0,0,0,0,33
1,1,1,1,2011,FOODS_3_226,CA_1,CA,0,0,0,0,13
2,1,1,1,2011,FOODS_3_228,CA_1,CA,0,0,0,0,14
3,1,1,1,2011,FOODS_3_252,CA_1,CA,0,0,0,0,19
4,1,1,1,2011,FOODS_3_377,CA_1,CA,0,0,0,0,20
...,...,...,...,...,...,...,...,...,...,...,...,...
196895,1969,2,6,2016,FOODS_3_555,WI_3,WI,1,1,0,0,39
196896,1969,2,6,2016,FOODS_3_586,WI_3,WI,1,1,0,0,38
196897,1969,2,6,2016,FOODS_3_668,WI_3,WI,1,1,0,0,12
196898,1969,2,6,2016,FOODS_3_694,WI_3,WI,1,1,0,0,49


In [28]:
data = data.rename(columns={"wday": "weekday"})

In [29]:
data

,index,weekday,month,year,item_id,store_id,state_id,is_sporting_event,is_cultural_event,is_national_event,is_religious_event,demand
0,1,1,1,2011,FOODS_3_080,CA_1,CA,0,0,0,0,33
1,1,1,1,2011,FOODS_3_226,CA_1,CA,0,0,0,0,13
2,1,1,1,2011,FOODS_3_228,CA_1,CA,0,0,0,0,14
3,1,1,1,2011,FOODS_3_252,CA_1,CA,0,0,0,0,19
4,1,1,1,2011,FOODS_3_377,CA_1,CA,0,0,0,0,20
...,...,...,...,...,...,...,...,...,...,...,...,...
196895,1969,2,6,2016,FOODS_3_555,WI_3,WI,1,1,0,0,39
196896,1969,2,6,2016,FOODS_3_586,WI_3,WI,1,1,0,0,38
196897,1969,2,6,2016,FOODS_3_668,WI_3,WI,1,1,0,0,12
196898,1969,2,6,2016,FOODS_3_694,WI_3,WI,1,1,0,0,49


In [30]:
def _month_to_string(x):
    if x==1:
        return 'JAN'
    elif x==2:
        return 'FEB'
    elif x==3:
        return 'MAR'
    elif x==4:
        return 'APR'
    elif x==5:
        return 'MAY'
    elif x==6:
        return 'JUN'
    elif x==7:
        return 'JUL'
    elif x==8:
        return 'AUF'
    elif x==9:
        return 'SEP'
    elif x==10:
        return 'OCT'
    elif x==11:
        return 'NOC'
    else:
        return 'DEC'
    
def _day_to_string(x):
    if x==1:
        return 'MON'
    elif x==2:
        return 'TUE'
    elif x==3:
        return 'WED'
    elif x==4:
        return 'THU'
    elif x==5:
        return 'FRI'
    elif x==6:
        return 'SAT'
    else:
        return 'SUN'

In [31]:
data['month'] = data['month'].apply(_month_to_string)
data['weekday'] = data['weekday'].apply(_day_to_string)

In [32]:
data

,index,weekday,month,year,item_id,store_id,state_id,is_sporting_event,is_cultural_event,is_national_event,is_religious_event,demand
0,1,MON,JAN,2011,FOODS_3_080,CA_1,CA,0,0,0,0,33
1,1,MON,JAN,2011,FOODS_3_226,CA_1,CA,0,0,0,0,13
2,1,MON,JAN,2011,FOODS_3_228,CA_1,CA,0,0,0,0,14
3,1,MON,JAN,2011,FOODS_3_252,CA_1,CA,0,0,0,0,19
4,1,MON,JAN,2011,FOODS_3_377,CA_1,CA,0,0,0,0,20
...,...,...,...,...,...,...,...,...,...,...,...,...
196895,1969,TUE,JUN,2016,FOODS_3_555,WI_3,WI,1,1,0,0,39
196896,1969,TUE,JUN,2016,FOODS_3_586,WI_3,WI,1,1,0,0,38
196897,1969,TUE,JUN,2016,FOODS_3_668,WI_3,WI,1,1,0,0,12
196898,1969,TUE,JUN,2016,FOODS_3_694,WI_3,WI,1,1,0,0,49


## Add Snap feature

SNAP are benefits for low income Americans to spend on food - payment takes place on different days depending on state and other local factors

In [33]:
snap = pd.concat([pd.DataFrame({'is_snap_day':calendar.snap_CA, 'state':'CA'}), 
                 pd.DataFrame({'is_snap_day':calendar.snap_TX, 'state':'TX'}), 
                 pd.DataFrame({'is_snap_day':calendar.snap_WI, 'state':'WI'})])

snap.reset_index(inplace=True)
snap['index'] = snap['index']+1

In [34]:
snap

,index,is_snap_day,state
0,1,0,CA
1,2,0,CA
2,3,0,CA
3,4,1,CA
4,5,1,CA
...,...,...,...
5902,1965,1,WI
5903,1966,0,WI
5904,1967,0,WI
5905,1968,0,WI


In [35]:
data = data.merge(snap, left_on=['state_id', 'index'], right_on=['state', 'index'])

In [62]:
data.drop(columns=['state_id', 'state'], inplace=True)

## Add lag features

In [64]:
#split in X and y 
y = pd.DataFrame(data['demand'])
X = data.drop(columns=['demand'])

In [65]:
X

,index,weekday,month,year,item_id,store_id,is_sporting_event,is_cultural_event,is_national_event,is_religious_event,is_snap_day
0,1,MON,JAN,2011,FOODS_3_080,CA_1,0,0,0,0,0
1,1,MON,JAN,2011,FOODS_3_226,CA_1,0,0,0,0,0
2,1,MON,JAN,2011,FOODS_3_228,CA_1,0,0,0,0,0
3,1,MON,JAN,2011,FOODS_3_252,CA_1,0,0,0,0,0
4,1,MON,JAN,2011,FOODS_3_377,CA_1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
196895,1969,TUE,JUN,2016,FOODS_3_555,WI_3,1,1,0,0,0
196896,1969,TUE,JUN,2016,FOODS_3_586,WI_3,1,1,0,0,0
196897,1969,TUE,JUN,2016,FOODS_3_668,WI_3,1,1,0,0,0
196898,1969,TUE,JUN,2016,FOODS_3_694,WI_3,1,1,0,0,0


In [39]:
y

,demand
0,33
1,13
2,14
3,19
4,20
...,...
196895,39
196896,38
196897,12
196898,49


In [57]:
# set lag features
fc_parameters = MinimalFCParameters()

In [58]:
# delete length features
del fc_parameters['length']

In [59]:
# print all lag features
print(fc_parameters)

{'sum_values': None, 'median': None, 'mean': None, 'standard_deviation': None, 'variance': None, 'root_mean_square': None, 'maximum': None, 'absolute_maximum': None, 'minimum': None}


In [60]:
def create_features(X, y, column_id, column_sort, feature_dict, time_windows):
    """
    Create lag features for y and add them to X

    Parameters:
    -----------
    X: pandas.DataFrame 
    feature matrix to which TS features are added.

    y: pandas.DataFrame, 
    time series to compute the features for.

    column_id: list, 
    list of column names to group by, e.g. ["shop","product"]. If set to None, 
    either there should be nothing to groupby or each group should be 
    represented by a separate target column in y. 

    column_sort: str,
    column name used to sort the DataFrame. If None, will be filled by an 
    increasing number, meaning that the order of the passed dataframes are used 
    as “time” for the time series.

    feature_dict: dict,
    dictionary containing feature calculator names with the corresponding 
    parameters

    time_windows : list of tuples, 
    each tuple (min_timeshift, max_timeshift), represents the time shifts for 
    ech time windows to comupute e.g. [(7,7),(1,14)] for two time windos 
    a) time window with a fix size of 7 and b) time window that starts with size
    1 and increases up to 14. Then shifts by 1 for each step. 
    """

    if column_id == None:
        X['id'] = 1

    else:
        X['id'] = X[column_id].astype(str).agg('_'.join, axis=1)

    if column_sort == None:
        X['time'] = range(X.shape[0])  

    else:
        X["time"] = X[column_sort]

    y["time"] = X["time"]
    y["id"] = X["id"]

    X = X.set_index(['id', 'time'])

    for window in time_windows:

        # create time series for given time window 
        df_rolled = roll_time_series(y, column_id="id", column_sort="time", 
                                     min_timeshift= window[0]-1, 
                                     max_timeshift= window[1]-1)

        # create lag features for given time window 
        df_features = extract_features(df_rolled, column_id="id", 
                                       column_sort="time",
                                       default_fc_parameters=feature_dict)

        # Add time window to feature name for clarification 
        feature_names = df_features.columns.to_list()
        feature_names = [name+"_"+str(window[1]) for name in feature_names]
        df_features.columns = feature_names

        # add features for given time window to feature matrix temp
        X = pd.concat([X,df_features],axis=1)

    y = y.set_index(['id', 'time'])
    y_column_names = y.columns.to_list()

    df = pd.concat([X,y],axis=1)
    df = df.dropna()
    df = df.reset_index(drop=True)

    y = df[y_column_names]
    X = df.drop(y_column_names, axis=1)

    return X, y

In [66]:
X

,index,weekday,month,year,item_id,store_id,is_sporting_event,is_cultural_event,is_national_event,is_religious_event,is_snap_day
0,1,MON,JAN,2011,FOODS_3_080,CA_1,0,0,0,0,0
1,1,MON,JAN,2011,FOODS_3_226,CA_1,0,0,0,0,0
2,1,MON,JAN,2011,FOODS_3_228,CA_1,0,0,0,0,0
3,1,MON,JAN,2011,FOODS_3_252,CA_1,0,0,0,0,0
4,1,MON,JAN,2011,FOODS_3_377,CA_1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
196895,1969,TUE,JUN,2016,FOODS_3_555,WI_3,1,1,0,0,0
196896,1969,TUE,JUN,2016,FOODS_3_586,WI_3,1,1,0,0,0
196897,1969,TUE,JUN,2016,FOODS_3_668,WI_3,1,1,0,0,0
196898,1969,TUE,JUN,2016,FOODS_3_694,WI_3,1,1,0,0,0


In [67]:
X, y  = create_features(X=X, y=y, column_id=['item_id',"store_id"], column_sort='index', 
                        feature_dict=fc_parameters, time_windows = [(7,7),(14,14),(28,28)])

Rolling: 100%|██████████████████████████████████| 20/20 [00:23<00:00,  1.16s/it]
/Users/aphil/opt/anaconda3/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/Users/aphil/opt/anaconda3/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/Users/aphil/opt/anaconda3/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/Users/aphil/opt/anaconda3/lib/python3.8/site-packag

In [70]:
X.drop(columns=["index"],inplace=True)

In [ ]:
 = pd.concat([X,y], axis=1)

In [ ]:
m5.to_csv("m5_final.csv", index=False)

In [ ]:
m5